In [4]:
import os
from filemanager import file_ext_search as fes
from dataclasses import dataclass, field
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

from pyproj import Proj, CRS, Transformer

In [5]:
@dataclass
class SegyPosFile:
    name: str
    path: str
    fin_traceno: int = 0
    datetime: list = field(default_factory=list)
    traceno: list = field(default_factory=list)
    cdp_x: list = field(default_factory=list)
    cdp_x_smoothed: list = field(default_factory=list)
    cdp_y_smoothed: list = field(default_factory=list)
    cdp_x_cartesian_smoothed: list = field(default_factory=list)
    cdp_y_cartesian_smoothed: list = field(default_factory=list)
    cdp_y: list = field(default_factory=list)
    year: list = field(default_factory=list)
    day: list = field(default_factory=list)
    hour: list = field(default_factory=list)
    minute: list = field(default_factory=list)
    second: list = field(default_factory=list)

In [6]:
def read_segypos(pos_files, finedict, baddict, posobj_list, year, utm_coords=False):
    for pos_file in pos_files:
        segy_name = os.path.splitext(os.path.basename(pos_file))[0]
        pos_obj = SegyPosFile(name=segy_name, path=pos_file)
        
        number = 0
        has_error = False
        was_before = False
        
        with open(pos_file, 'r') as file1:
            file_content = file1.read().splitlines()
            
            for line in file_content[1:]:
                line_content = line.split()

crs_wgs84 = CRS.from_epsg(4326)
crs_utm35n = CRS.from_epsg(32635)
crs_utm34n = CRS.from_epsg(32634)

transformer = Transformer.from_crs(crs_wgs84, crs_utm34n, always_xy=True)

In [7]:
from sbp_tools.sbp_track_processing import process_track, save_track_to_radex, save_track

crs_wgs84 = CRS.from_epsg(4326)
crs_utm35n = CRS.from_epsg(32635)
crs_utm34n = CRS.from_epsg(32634)

transformer = Transformer.from_crs(crs_wgs84, crs_utm34n, always_xy=True)

data_path = r'C:\Data\ABP56_FZ_UTM35N\sgy_ps3_SLF_FIXED'
# track
save_to = r'C:\Data\ABP56_FZ_UTM35N\abp56_utm35n_track.txt'

is_utm = True
year = 2024
pos_files = fes.file_ext_search('.txt', data_path, recursive=True)
print(len(pos_files))

segy_pos_objs = []
bad_data_dict = {}
fine_data_dict = {}

read_segypos(pos_files, fine_data_dict, bad_data_dict, segy_pos_objs, year, utm_coords = is_utm)
# process_track(segy_pos_objs, transformer, window_length=201, smooth=False, utm_coords=is_utm)
save_track(segy_pos_objs, save_to)
# save_track_to_radex(segy_pos_objs)

Searching *.txt files in directory:C:\Data\ABP56_FZ_UTM35N\sgy_ps3_SLF_FIXED
360
